## XGBoost with RAPIDS

In this notebook, we will show how to work with GPU accelerated XGBoost in RAPIDS with the dataset we just created.

**Table of Contents**

* [Import Libraries](#Import-Libraries)
* [XGBoost Training](#XGBoost-Training)
    * [Define Parameters](#Define-Parameters)
    * [Train XGBoost Model](#Train-XGBoost-Model)

### Import Libraries

In [1]:

import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import sklearn; print('Scikit-Learn Version:', sklearn.__version__)
import xgboost; print('XGBoost Version:', xgboost.__version__)
from sklearn.metrics import accuracy_score

import time 


import rapids_lib_v8 as rl
''' NOTE: anytime changes are made to rapids_lib.py you can either:
      1. refresh/reload via the code below, OR
      2. restart the kernel '''
import importlib; importlib.reload(rl)

numpy Version: 1.16.2
pandas Version: 0.24.2
Scikit-Learn Version: 0.20.4
XGBoost Version: 0.90.rapidsdev1


<module 'rapids_lib_v8' from '/rapids/notebooks/ml_tutorial/version_101/rapids_lib_v8.py'>

In the previous notebook we already created our dataset and made it ready for training and testing .
Remember, we intentionally named the objects we created in a way you can easy distingwish whether you are using CPU based libraries such as pandasDataFrame(`_pDF`) vs GPU library cuDF (`_cDF`)

Let's first load the variables we stored in the previous notebook. 


In [2]:
%store -r trainData_cDF
%store -r trainLabels_cDF
%store -r testData_cDF 
%store -r testLabels_cDF

In [3]:
%store -r expLog
%store -r trainData_pDF
%store -r trainLabels_pDF 
%store -r testData_pDF
%store -r testLabels_pDF

### XGBoost Training 


I like to say that we have  three main components here:

<font color=green>__1. Data to DMatrix format__</font>


In order for XGBoost to be able to use our data, we’ll need to transform it into a specific format that XGBoost can handle. That format is called **DMatrix**. It’s a very simple one-linear to transform a numpy array of data to DMatrix format.

We can instantiate an object of the `xgboost.DMatrix` by passing in the feature matrix as the first argument followed by the label vector.

`D_train = xgboost.DMatrix(X_train, label=Y_train)
 D_test = xgboost.DMatrix(X_test, label=Y_test)`
 
<font color=green>__2. Defining the parameters__</font> 

Now that our data is all loaded up, we can define the **parameters**. We’ve set up some of the most important ones below to get us started. For more complicated tasks and models, the full list of possible parameters is available on the official [XGBoost website](https://xgboost.readthedocs.io/en/latest/parameter.html)

<font color=green>__3. Training and testing__<font/>

We can finally train our model:
`model = xgboost.train(param, D_train, steps)`



#### Define Parameters

Let's separate the parameters we are using when training CPU vs GPU based XGboost. Take some time to see that almost everything is the same besides 2 parameters:

- `tree_method` : we will use `gpu_hist` for GPU training
- `n_gpus vs n_jobs` : instead of specifying how many CPU cores our training will use, for GPU based XGboost we will specify number of GPUs we are using. 

"`-1`" means *use all the avaialble GPUs you have*

Note: *Hinge loss* penalizes incorrectly classified examples and correctly classified examples that lie within the margin(correct but not confident)
Hinge Lost is faster to train via gradient decent, easier to compute than cross entropy (log loss).  


In [4]:
nCores = !nproc --all
nCores = int(nCores[0]) # we want to extract number of cores the CPU has 

paramsCPU = {
    'max_depth': 10,
    'learning_rate': .1,
    'num_boost_rounds': 100,
    'lambda': 1,
    'objective': 'binary:hinge',
    'tree_method': 'hist',
    'n_jobs': nCores,
    'random_state': 0
}

paramsGPU = {
    'max_depth': 10,
    'learning_rate': .1,
    'num_boost_rounds': 100,
    'lambda': 1,
    'objective': 'binary:hinge',
    'tree_method': 'gpu_hist',
    'n_gpus': 1,    
    'random_state': 0
}

#### Train XGBoost Model 

**Monitor gpu utilization** by typing `watch 0.1 nvidia-smi` in your console or use our GPU monitoring tool in System Dashboards. 
You can find it on the left side of our notebook. I recommend to organize your Dashboard in a separate window to look something like this :



![System Monitoring Tool](monitoring.JPG)

Now when we have our parameters defined, let's use two functions below to:
1. pass the parameters
2.convert data to Dmatrix and
3.complete the training and testing 

You can see that win both cases (GPU and CPU) we need to pass pandas dataframe to DMatrix.

In [5]:
''' -------------------------------------------------------------------------
>  CPU Train and Test
------------------------------------------------------------------------- '''
def train_model_CPU (trainData_pDF, testData_pDF, paramsCPU = {}):    
    print('training xgboost model on {} CPU cores'.format(nCores) )
    
    "we will pass the Pandas DataFrame to DMatrix"
    trainDMatrix = xgboost.DMatrix( trainData_pDF, label = trainLabels_pDF)
    startTime = time.time()
    xgBoostModelCPU = xgboost.train( dtrain = trainDMatrix, params = paramsCPU,
                                   num_boost_round = paramsCPU['num_boost_rounds'])
    cpuTrainingTime = time.time()-startTime
    print("CPU training time:" , cpuTrainingTime,"seconds.")
    return xgBoostModelCPU, cpuTrainingTime

def test_model_CPU ( trainedModelCPU, testData_pDF, testLabels_pDF ):
    print('testing xgboost model on CPU')
    testDMatrix = xgboost.DMatrix( testData_pDF, label=testLabels_pDF)
    startTime = time.time()    

    predictionsCPU = trainedModelCPU.predict(testDMatrix)
    cpuTestingTime = time.time()-startTime
    print("CPU testing time:" ,cpuTestingTime, "seconds.")
    return predictionsCPU, cpuTestingTime


In [6]:
#model training
trainedModelCPU, t_trainCPU = train_model_CPU ( trainData_pDF, trainLabels_pDF, paramsCPU )
#model inferencing
predictionsCPU, t_inferCPU = test_model_CPU ( trainedModelCPU, testData_pDF, testLabels_pDF )

training xgboost model on 40 CPU cores
CPU training time: 915.9243884086609 seconds.
testing xgboost model on CPU
CPU testing time: 0.05548715591430664 seconds.


In [7]:
expLog = rl.update_log( expLog, [['CPU_model_training', t_trainCPU], ['CPU_model_inference', t_inferCPU]] ); 

 + adding log entry [ CPU_model_training       : 915.92439 s ]
 + adding log entry [ CPU_model_inference      :   0.05549 s ]


In [8]:
''' -------------------------------------------------------------------------
>  GPU Train and Test
------------------------------------------------------------------------- '''
def train_model_GPU (trainData_cDF, testData_cDF, paramsGPU = {}):    
    print('training xgboost model on GPU');  
    startTime = time.time() 
    trainDMatrix = xgboost.DMatrix( trainData_cDF, label = trainLabels_cDF)
    
    trainedModelGPU = xgboost.train( dtrain = trainDMatrix, params = paramsGPU, num_boost_round = paramsGPU['num_boost_rounds'] )
    gpuTrainingTime = time.time()-startTime
    
    print("GPU training time:" ,gpuTrainingTime,"seconds")
    return trainedModelGPU, gpuTrainingTime

def test_model_GPU ( trainedModelGPU, testData_cDF, testLabels_cDF ):
    print('testing xgboost model on GPU')
    startTime = time.time()   
    testDMatrix = xgboost.DMatrix( testData_cDF, label=testLabels_cDF)
    
    predictionsGPU = trainedModelGPU.predict(testDMatrix)
    gpuTestingTime = time.time()-startTime
    print("GPU testing time:" ,gpuTestingTime, "seconds")
    return predictionsGPU, gpuTestingTime


In [9]:
#model training
trainedModelGPU, t_trainGPU = train_model_GPU ( trainData_cDF, trainLabels_cDF, paramsGPU )
# model inferencing
predictionsGPU, t_inferGPU = test_model_GPU ( trainedModelGPU, testData_cDF, testLabels_cDF)

training xgboost model on GPU
GPU training time: 2.4469590187072754 seconds
testing xgboost model on GPU
GPU testing time: 0.005692958831787109 seconds


In [10]:
gpu_accuracy = accuracy_score(testLabels_pDF,predictionsGPU);print(gpu_accuracy)

0.9433875


In [11]:
cpu_accuracy = accuracy_score(testLabels_pDF,predictionsCPU);print(cpu_accuracy)

0.9466375


In [12]:
%store trainData_cDF
%store trainLabels_cDF
%store testData_cDF 
%store testLabels_cDF

%store expLog
%store trainData_pDF
%store trainLabels_pDF 
%store testData_pDF
%store testLabels_pDF

Stored 'trainData_cDF' (DataFrame)
Stored 'trainLabels_cDF' (DataFrame)
Stored 'testData_cDF' (DataFrame)
Stored 'testLabels_cDF' (DataFrame)
Stored 'expLog' (dict)
Stored 'trainData_pDF' (DataFrame)
Stored 'trainLabels_pDF' (DataFrame)
Stored 'testData_pDF' (DataFrame)
Stored 'testLabels_pDF' (DataFrame)
